In [ ]:
#appendix
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os
import string
import glob
import pickle
import random
import sys
from collections import defaultdict, Counter

from keras import backend as K
from keras import initializers, regularizers, constraints
from keras import losses
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Bidirectional, GRU, TimeDistributed, Conv1D, MaxPooling1D, Embedding
from keras.layers import SeparableConv2D, MaxPooling2D, Dropout,concatenate, SpatialDropout1D, SeparableConv1D, Activation, Reshape
from keras_self_attention import SeqSelfAttention   
from keras.models import Model, Sequential, load_model
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.engine import Layer

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

from wikipedia2vec import Wikipedia2Vec
from wikipedia2vec.dictionary import Dictionary  
from wikipedia2vec.mention_db import MentionDB 
from wikipedia2vec.utils.tokenizer.mecab_tokenizer import MeCabTokenizer
from wikipedia2vec.utils.tokenizer.regexp_tokenizer import RegexpTokenizer

from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session

In [14]:
n = 10
MAX_SENTENCE_NUM = n # maximum number of sentences 
EMBED_SIZE = 200      # vector size of word embedding

# for META
ENT_EMBED_SIZE = 100 # vector size of co-occuring entities (wikipedia2vec)
MAX_ENT_NUM = 5 * n # maximum number of co-occuring entities
MAX_META_FEATURES = 20000 # maximum number of meta features

# for ENT
MAX_CHAR_LENGTH = 30  # maximum number of characters in each entity

# for CONTEXT
MAX_FEATURES = 500000 # maximum number of unique words that should be included in the tokenized word index
MAX_WORD_NUM = 35     # maximum number of words in each sentence

In [15]:
# Context selection model

c_embedding_layer = Embedding(len(train_word_index) + 1,EMBED_SIZE,weights=[embedding_matrix], 
                            input_length=MAX_WORD_NUM, trainable=False,name='c_word_embedding')

# Words level attention model
c_word_input = Input(shape=(MAX_WORD_NUM,), dtype='int32',name='c_word_input')
c_word_sequences = c_embedding_layer(c_word_input)

c_conc = Bidirectional(GRU(units=256, return_sequences=True, activation='relu'))(c_word_sequences)

c_word_att = SeqSelfAttention(attention_activation='sigmoid')(c_conc)
c_sent = Flatten()(c_word_att)
c_drop = Dropout(0.5, name='dropout')(c_sent)
c_preds = Dense(label_num, activation='softmax',name='output')(c_drop)

cs_model = Model(inputs = c_word_input, outputs = c_preds)

In [16]:
print(cs_model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c_word_input (InputLayer)    (None, 35)                0         
_________________________________________________________________
c_word_embedding (Embedding) (None, 35, 200)           79159800  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 35, 512)           701952    
_________________________________________________________________
seq_self_attention_4 (SeqSel (None, 35, 512)           32833     
_________________________________________________________________
flatten_4 (Flatten)          (None, 17920)             0         
_________________________________________________________________
dropout (Dropout)            (None, 17920)             0         
_________________________________________________________________
output (Dense)               (None, 185)               3315

In [17]:
# Model compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss', patience=0)]

In [18]:
# Entity typing model

#--------------META--------------------
# urls and user names
m_word_input = Input(shape=(MAX_META_FEATURES,), dtype='float32',name='m_word_input')
m_dense1 = Dense(256, activation='relu', name='m_word_dense1')(m_word_input) 

# co-occuring entities
m_ent_input = Input(shape=(MAX_ENT_NUM, ENT_EMBED_SIZE), dtype='float32',name='m_ent_input')
m_ent_att = SeqSelfAttention(attention_activation='sigmoid', attention_width=15)(m_ent_input)
m_ent_att_ = Flatten()(m_ent_att)

m_flat = concatenate([m_dense1, m_ent_att_], axis=1)

#--------------ENT------------------------
# character encoding
e_embedding_layer = Embedding(len(train_echar_index) + 1,16,embeddings_initializer='glorot_uniform', 
                            input_length=MAX_CHAR_LENGTH, trainable=True,name='e_char_embedding')
e_char_input = Input(shape=(MAX_CHAR_LENGTH,), dtype='int32',name='e_char_input')
e_char_sequences = e_embedding_layer(e_char_input)
e_conc = Bidirectional(GRU(units=64, return_sequences=False, activation='relu'), merge_mode='concat')(e_char_sequences)

# token encoding
e_word_input = Input(shape=(EMBED_SIZE,), dtype='float32',name='e_word_input')

e_flat = concatenate([e_conc, e_word_input], axis=1)

#------------CONTEXT---------------------
c_embedding_layer = Embedding(len(train_word_index) + 1,EMBED_SIZE,weights=[embedding_matrix], 
                            input_length=MAX_WORD_NUM, trainable=False,name='c_word_embedding')

# Words level attention model
c_word_input = Input(shape=(MAX_WORD_NUM,), dtype='int32',name='c_word_input')
c_word_sequences = c_embedding_layer(c_word_input)
c_conc = Bidirectional(GRU(units=256, return_sequences=True, activation='relu'))(c_word_sequences)
c_word_att = SeqSelfAttention(attention_activation='sigmoid')(c_conc)
c_sent = Flatten()(c_word_att)
c_wordEncoder = Model(inputs = c_word_input,outputs = c_sent)

# Sentence level attention model
c_sent_input = Input(shape=(MAX_SENTENCE_NUM,MAX_WORD_NUM), dtype='int32',name='c_sent_input')
c_sent_encoder = TimeDistributed(c_wordEncoder,name='c_sent_linking')(c_sent_input)
c_sent_att = SeqSelfAttention(attention_activation='sigmoid', attention_width=n)(c_sent_encoder)
c_flat = Flatten()(c_sent_att)

flat = concatenate([m_flat, e_flat, c_flat], axis=1)
flat_ = Dropout(0.5, name='dropout')(flat)
preds = Dense(label_num, activation='softmax',name='output2')(flat_)

et_model = Model([m_word_input, m_ent_input, e_char_input, e_word_input, c_sent_input], preds)

In [19]:
print(c_wordEncoder.summary())
print("")
print(et_model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c_word_input (InputLayer)    (None, 35)                0         
_________________________________________________________________
c_word_embedding (Embedding) (None, 35, 200)           79159800  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 35, 512)           701952    
_________________________________________________________________
seq_self_attention_6 (SeqSel (None, 35, 512)           32833     
_________________________________________________________________
flatten_6 (Flatten)          (None, 17920)             0         
Total params: 79,894,585
Trainable params: 734,785
Non-trainable params: 79,159,800
_________________________________________________________________
None

Model: "model_5"
____________________________________________________________________________________

In [ ]:
# Model compile
model.compile(loss={'output2': 'categorical_crossentropy'},optimizer='adam',metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss', patience=0)]
history = model.fit([ef_train,ee_train,ec_train, ew_train,x_train], y_train, validation_data=([ef_devel, ee_devel, ec_devel, ew_devel, x_devel], y_devel), epochs=20, shuffle=False, batch_size=32, callbacks=callbacks)
model.save('model_cross_.h5', include_optimizer=False)

#-Predict and print result
# y_pred = model.predict([ef_test, ee_test, ec_test, ew_test, x_test])
# print_result(y_test, y_pred, n)

# y_pred = model.predict([ef_test_p, ee_test_p, ec_test_p, ew_test_p, x_test_p])
# print_result(y_test, y_pred, n)

# y_pred = model.predict([ef_test_amb, ee_test_amb, ec_test_amb, ew_test_amb, x_test_amb])
# print_result(y_test_amb, y_pred, n)

# y_pred = model.predict([ef_test_amb_p, ee_test_amb_p, ec_test_amb_p, ew_test_amb_p, x_test_amb_p])
# print_result(y_test_amb, y_pred, n)
